In [136]:
import pandas as pd
from fuzzywuzzy import fuzz
from nameparser import HumanName

In [12]:
ppd = pd.read_csv('../../../Data/PPD/ppd_data_20200404.csv')
spec = pd.read_csv('../../../Data/PPD/speciality_id.csv')
df = pd.read_csv('../../../Data/Medscape/Memorium_USA.csv')

In [137]:
ppd

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102000021,A,1,3.0,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,NaN,...,NC,27617,7408.0,R077,NaN,NaN,NaN,NaN,NaN,NaN
2,102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262549,96501960028,A,1,2.0,ROBERT SAKULANDA MD,SAKULANDA,ROBERT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262550,96501960036,A,1,1.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,ST RITA'S MEDICAL CENTER,...,OH,45801,4667.0,C005,NaN,NaN,NaN,NaN,NaN,NaN
1262551,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,VA,24541,2987.0,C002,NaN,NaN,NaN,NaN,NaN,NaN
1262552,96501960052,A,1,2.0,MASIKU A MDALA-GAUSI MD,MDALA-GAUSI,MASIKU,AGATHA,NaN,NaN,...,NC,27157,1.0,C000,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
PPD_SPEC = pd.merge(ppd, spec, left_on='PRIM_SPEC_CD', right_on='SPEC_CD')

In [20]:
def split_names(roster_df):
    '''Splits name column into components'''
    roster_df = roster_df.drop_duplicates()
    dict_list = []
    for row in roster_df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
            'NAME':row.NAME,
            'FIRST_NAME': name_parsed.first.upper(),
            'LAST_NAME': name_parsed.last.upper(),
            'MIDDLE_NAME': name_parsed.middle.upper(),
            'SUFFIX':name_parsed.suffix.upper(),
            'NICKNAME':name_parsed.nickname.upper(),
            'TITLE':name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, roster_df, on='NAME')
    return new_df

In [22]:
df = split_names(df)

In [30]:
bad_spec_words = [
    'Nurse',
    'Vet',
    'Transport',
    'Assistant',
    'Receptionist'
]

In [65]:
mes =[]
for row in df.itertuples():
    NAME = row.NAME
    ME = 'None'
    keep = True
    for word in bad_spec_words:
        if word in row.SPECIALTY:
            keep = False
    if keep == True:
        new_df = PPD_SPEC[(PPD_SPEC.FIRST_NAME==row.FIRST_NAME)&(PPD_SPEC.LAST_NAME==row.LAST_NAME)]
        if len(new_df)==0:
            pass
        elif len(new_df)>1:
            new_df = new_df[new_df.BIRTH_YEAR.isin([2019.0-int(row.AGE), 2020.0-int(row.AGE)])]
            if len(new_df)>1:
                print('wtf')
        if len(new_df)==1:
            ME = list(new_df.ME)[0]
    mes.append(ME)

In [66]:
df['ME']=mes

In [69]:
df[df.ME!='None']

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,AGE,SPECIALTY,CITY,STATE,COUNTRY,LOCATION,ME
6,Marques Bolden,MARQUES,BOLDEN,,,,,35,Hospitalist,South Bend,Indiana,USA,None,1720102877
15,James T. Goodrich,JAMES,GOODRICH,T.,,,,73,Neurosurgeon Director of the Division of Pedia...,New York City,New York,USA,Pediatrics,3501801606
17,Alex Hsu,ALEX,HSU,,,,,67,Internal Medicine,Margate,Florida,USA,Northwest Medical Center,1401820236
25,John F. Murray,JOHN,MURRAY,F.,,,,92,Pulmonary and Critical Care Physician,San Francisco,California,USA,San Francisco General Hospital,511530385
27,Tomas Pattugalan,TOMAS,PATTUGALAN,,,,,70,Internal Medicine Physician,Queens,New York,USA,None,74810740365
28,Mark Respler,MARK,RESPLER,,,,,66,Urologist,Brooklyn,New York,USA,None,3508801512
30,Stephen Schwartz,STEPHEN,SCHWARTZ,,,,,78,Pathologist,Seattle,Washington,USA,None,2405670502


In [70]:
PPD_SPEC.to_csv('../../../Data/PPD/ppd_spec.csv',index=False)

In [130]:
PPD_SPEC[PPD_SPEC.ME==3005731398]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,SPEC_CD,SPEC_ID,DESC
945271,3005731398,A,1,2.0,DAVID A WOLIN MD,WOLIN,DAVID,ALLEN,NaN,NaN,...,C003,NaN,NaN,NaN,NaN,NaN,NaN,R,1853,RADIOLOGY


In [72]:
ppd

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102000021,A,1,3.0,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,NaN,...,NC,27617,7408.0,R077,NaN,NaN,NaN,NaN,NaN,NaN
2,102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262549,96501960028,A,1,2.0,ROBERT SAKULANDA MD,SAKULANDA,ROBERT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262550,96501960036,A,1,1.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,ST RITA'S MEDICAL CENTER,...,OH,45801,4667.0,C005,NaN,NaN,NaN,NaN,NaN,NaN
1262551,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,VA,24541,2987.0,C002,NaN,NaN,NaN,NaN,NaN,NaN
1262552,96501960052,A,1,2.0,MASIKU A MDALA-GAUSI MD,MDALA-GAUSI,MASIKU,AGATHA,NaN,NaN,...,NC,27157,1.0,C000,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
PPD_SPEC[(PPD_SPEC.LAST_NAME=='DEJESUS')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,SPEC_CD,SPEC_ID,DESC
15478,1875901261,A,1,1.0,PAULA W DEJESUS DO,DEJESUS,PAULA,WILLOUGHBY,NaN,GREAT RIVER MEDICAL CENTER,...,R004,WILLOUGHBY,J,PAULA,NaN,NaN,NaN,EM,1682,EMERGENCY MEDICINE
18639,2307171646,A,1,2.0,JULIO DEJESUS MD,DEJESUS,JULIO,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM,1682,EMERGENCY MEDICINE
92060,3519131136,A,1,1.0,JOHANNA M DEJESUS MD,DEJESUS,JOHANNA,M,NaN,BREVARD HEALTH ALLIANCE,...,C007,PINEIRO,MARISOL,JOHANNA,NaN,NaN,NaN,PD,1800,PEDIATRICS
286318,74810920380,A,1,2.0,VICTORINO L DEJESUS MD,DEJESUS,VICTORINO,LOPEZ,NaN,NaN,...,C007,NaN,NaN,NaN,NaN,NaN,NaN,IM,1724,INTERNAL MEDICINE
413982,3546121703,A,1,2.0,GARRETT L DEJESUS MD,DEJESUS,GARRETT,LI,NaN,NaN,...,C011,NaN,NaN,NaN,NaN,NaN,NaN,DR,1679,DIAGNOSTIC RADIOLOGY
419943,4201990373,A,1,2.0,RICARDO DEJESUS MD,DEJESUS,RICARDO,NaN,NaN,URB PALMA REAL,...,C003,NaN,NaN,NaN,NaN,NaN,NaN,DR,1679,DIAGNOSTIC RADIOLOGY
499188,1176120050,A,1,2.0,EE-LENG DEJESUS DO,DEJESUS,EE-LENG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FM,1691,FAMILY MEDICINE
607785,65401091658,A,1,1.0,JENNIFER L DEJESUS MD,DEJESUS,JENNIFER,LYDIA,NaN,NaN,...,C030,NaN,NaN,NaN,NaN,NaN,NaN,FM,1691,FAMILY MEDICINE
697933,3548050757,A,1,2.0,NIDIA H DEJESUS MD,DEJESUS,NIDIA,HAYDEE,NaN,NaN,...,C035,NaN,NaN,NaN,NaN,NaN,NaN,PS,1841,PLASTIC SURGERY
705051,519120518,A,1,1.0,JUAN M DEJESUS JR MD,DEJESUS,JUAN,MANUEL,2.0,BLDG 500 RM E2322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,1793,PSYCHIATRY


In [78]:
PPD_SPEC[(PPD_SPEC.LAST_NAME=='SAFELY')]['BIRTH_YEAR']

335530    1962.0
Name: BIRTH_YEAR, dtype: float64

In [77]:
PPD_SPEC.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [91]:
for row in ppd[ppd.LAST_NAME=='SAFLEY'].itertuples():
    print(fuzz.ratio(row.FIRST_NAME, 'CHARLIE'))

36
31
33
31
14
29
31
29
36
55


In [139]:
PPD_SPEC[(PPD_SPEC.LAST_NAME=='CALDERA NIEVES')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST,SPEC_CD,SPEC_ID,DESC
332165,4201831399,A,1,2.0,LUIS A CALDERA NIEVES MD,CALDERA NIEVES,LUIS,A,NaN,NaN,...,C076,CALDERA,A,LUIS,NaN,NaN,NaN,OBG,1769,OBSTETRICS & GYNECOLOGY


In [89]:
fuzz.ratio('CHAS','CHARLIE')

55

In [117]:
def fix_me(me_list):
    nums =[]
    for num in me_list:
        num=str(num)
        num = num.replace('.0','')
        if len(num)==10:
            num = '0'+num
        elif len(num)==9:
            num = '00'+num
        elif len(num)==8:
            num = '000'+num
        nums.append(num)
    return nums

def append_me(roster_df, PPD):
    '''Matches to PPD and appends ME'''
    data_split = split_names(roster_df)

    bad_spec_words = [
        'Nurse',
        'Vet',
        'Transport',
        'Assistant',
        'Receptionist'
    ]

    mes = []
    for row in data_split.itertuples():
        physician_me = 'None'
        keep = True
        for word in bad_spec_words:
            if word in row.SPECIALTY:
                keep = False
        if keep:
            try:
                years = [2019.0-int(row.AGE), 2020.0-int(row.AGE)]
            except ValueError:
                print('NO AGE')
                pass    
            new_df = PPD[(PPD.FIRST_NAME == row.FIRST_NAME)&(PPD.LAST_NAME == row.LAST_NAME)]
            if len(new_df) == 0:
                new_df = PPD[(PPD.LAST_NAME == row.LAST_NAME) & (PPD.BIRTH_YEAR.isin(years))]
                if len(new_df)==0:
                    pass
                if len(new_df)>1:
                    new_df = new_df[new_df.CITY==row.CITY.upper()]
            elif len(new_df) > 1:
                new_df = new_df[new_df.BIRTH_YEAR.isin(years)]
                if len(new_df) > 1:
                    print('wtf')
            if len(new_df) == 1:
                physician_me = list(new_df.ME)[0]
            
        mes.append(physician_me)

    data_split['ME'] = fix_me(mes)
    data_me = data_split[data_split.ME != 'None']
    return data_split, data_me

In [94]:
df = pd.read_csv('../../../Data/Medscape/Memorium_USA_2020-04-09.csv')

In [95]:
PPD = ppd

In [118]:
a, b = append_me(df, PPD_SPEC)

NO AGE


In [119]:
len(b)

13

In [120]:
b

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,AGE,SPECIALTY,CITY,STATE,COUNTRY,LOCATION,LINK,ME
1,Ashraf Abdo,ASHRAF,ABDO,,,,,60,General Practitioner,New York City,New York,USA,None,https://www.egypttoday.com/Article/1/83243/Egy...,91502920371
10,Ricardo Castaneda,RICARDO,CASTANEDA,,,,,64,Psychiatrist,New York City,New York,USA,None,None,42901790064
17,Jessie Ferreras,JESSIE,FERRERAS,,,,,55,Primary Care Physician,Waldwick,New York,USA,None,None,74801820887
19,Chris Firlit,CHRIS,FIRLIT,,,,,37,PGY6 Oral-Maxillofacial Surgery Chief Resident...,Detroit,Michigan,USA,None,https://www.detroitnews.com/story/business/202...,02507173333
21,Frank Gabrin,FRANK,GABRIN,,,,,60,Emergency Medicine Physician,New York City,New York,USA,None,https://www.nj.com/coronavirus/2020/04/nj-er-d...,04177851693
22,James T. Goodrich,JAMES,GOODRICH,T.,,,,73,Neurosurgeon Director of the Division of Pedia...,New York City,New York,USA,Pediatrics,https://www.nytimes.com/2020/04/01/obituaries/...,03501801606
25,Alex Hsu,ALEX,HSU,,,,,67,Internal Medicine,Margate,Florida,USA,Northwest Medical Center,https://www.wptv.com/news/coronavirus/dr-alex-...,01401820236
33,John F. Murray,JOHN,MURRAY,F.,,,,92,Pulmonary and Critical Care Physician,San Francisco,California,USA,San Francisco General Hospital,https://www.mercurynews.com/2020/03/25/longtim...,00511530385
35,Tomas Pattugalan,TOMAS,PATTUGALAN,,,,,70,Internal Medicine Physician,Queens,New York,USA,None,http://www.oleantimesherald.com/news/state/pri...,74810740365
36,Mark Respler,MARK,RESPLER,,,,,66,Urologist,Brooklyn,New York,USA,None,https://hamodia.com/2020/03/27/bde-dr-mark-all...,03508801512


In [107]:
PPD_SPEC.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [123]:
test = list(spec['DESC'])

In [129]:
for row in df.itertuples():
    best_score = 0
    for spec in test:
        score = fuzz.ratio(row.SPECIALTY, spec.title())
        if score>best_score:
            best_score=score
            chosen_spec = spec
    if best_score > 45:
        print(row.SPECIALTY)
        print(chosen_spec)
        print(best_score)
        print('---')

Founder Professor of Internal Medicine and Rheumatology Escuela de Medicina Luis Razetti
INTERNAL MEDICINE/PREVENTIVE MEDICINE
46
---
General Practitioner
GENERAL PRACTICE
83
---
Nurse
BURNS
60
---
Physician Assistant Emergency Medicine
PEDIATRICS/EMERGENCY MEDICINE
66
---
Resident Nurse
HEAD AND NECK SURGERY
46
---
Food and Nutrition Services
ADDICTION MEDICINE
53
---
Psychiatrist
PSYCHIATRY
82
---
Director of Radiology
PEDIATRIC RADIOLOGY
65
---
Nurse
BURNS
60
---
Nurse
BURNS
60
---
Primary Care Physician
PULMONARY & CRITICAL CARE MEDICINE
54
---
Licensed Practical Nurse
GENERAL PRACTICE
60
---
PGY6 Oral-Maxillofacial Surgery Chief Resident (presumed)
ORAL & MAXILLOFACIAL SURGERY
59
---
Receptionist
SEX EDUCATION
48
---
Emergency Medicine Physician
EMERGENCY MEDICINE
78
---
Physician Assistant
PHYSICAL MEDICINE AND REHABILITATION
51
---
Internal Medicine
INTERNAL MEDICINE
100
---
ER Nurse
BURNS
46
---
Pediatric Nurse,Kings County Hospital
PEDIATRIC TRANSPLANT HEPATOLOGY
53
---
Medica

In [132]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

today_date = datetime.today()
one_month_ago = today_date - relativedelta(months=1)
one_month_ago.strftime("%b" " " "%Y")

todays date time: 2020-04-10 09:41:22.297333
one month ago date time: 2020-03-10 09:41:22.297333
one month ago date: 2020-03-10


'Mar 2020'